In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import rlrom
from rlrom.app import *
from rlrom.testers import *
import rlrom.utils as hf

In [2]:
from datetime import timedelta
from pandas import DataFrame

from simglucose.simulation.sim_engine import batch_sim
from simglucose_init import PATIENT_NAMES, build_sim_obj




In [3]:
NUM_SCENARIOS = 1

def batch_simglucose() -> list[DataFrame]:
    perturb = True

    # Define meals as a list of tuples (time, meal_size) where time is the hour in a day in 24-hour format.
    meals = [(7, 45), (12, 70), (16, 15), (18, 80), (23, 10)]
    if perturb:
        # TODO different purterbation strategies
        meals.insert(5, (21, 0))  # add a dummy meal of size 0
        meals.insert(3, (14, 0))  # add a dummy meal of size 0
        meals.insert(1, (10, 0))  # add a dummy meal of size 0

    # Create scenarios
    sim_obj_list = [build_sim_obj(meals, name) for name in PATIENT_NAMES[:NUM_SCENARIOS]]

    # Batch simulation
    return batch_sim(sim_obj_list, parallel=True)




In [4]:
sim_results = batch_simglucose()


Process ID: 117284
Simulation starts ...
Simulation Completed!
Simulation took 2.7844395637512207 sec.


In [5]:
sim_results[0].to_csv('simglucose_results.csv')

In [6]:
print(sim_results[0])

                             BG         CGM  CHO   insulin      LBGI  \
Time                                                                   
2024-09-04 00:00:00  149.020000  165.793949  0.0  0.013933  0.000000   
2024-09-04 00:03:00  149.020000  161.582022  0.0  0.013933  0.000000   
2024-09-04 00:06:00  149.020000  159.239373  0.0  0.013933  0.000000   
2024-09-04 00:09:00  149.020000  157.472710  0.0  0.013933  0.000000   
2024-09-04 00:12:00  149.020000  156.108950  0.0  0.013933  0.000000   
...                         ...         ...  ...       ...       ...   
2024-09-04 23:48:00   93.366001   93.736518  0.0  0.013933  1.204979   
2024-09-04 23:51:00   93.288345   92.966818  0.0  0.013933  1.215732   
2024-09-04 23:54:00   93.210068   92.409334  0.0  0.013933  1.226629   
2024-09-04 23:57:00   93.133161   92.171035  0.0  0.013933  1.237391   
2024-09-05 00:00:00   93.059999   92.250490  NaN       NaN  1.247680   

                         HBGI      Risk  
Time                 

In [7]:
env   = "offline_simglucose"
model = "some_BB_controller"

tester.env = env
tester.model_id = model


In [8]:
# creates one mock run to begin with
trace_idx = 0
trace = []
total_reward = 0
for i in range(10):
    time_step = i
    obs= [i*3.14]
    action = [0]
    next_obs = [(i+1)*3.14]
    reward = 1
    total_reward += reward
    done = False

    
    state = [time_step, obs, action, next_obs, reward, done]
    trace.append(state)


In [9]:

trace_idx = tester.add_trace(trace)
new_record = pd.DataFrame({'trace_idx':trace_idx, 
                           'env_name':tester.env_name,
                           'model_name': [tester.model_id],
                           'seed': [trace_idx], 'total_reward': [total_reward]})

if tester.evals is None:
    tester.evals = new_record
else:
    tester.evals = pd.concat([tester.evals, new_record])

tester.trace_idx = trace_idx


In [10]:
print(trace_idx)

0


In [21]:
tester.signals_names = ["bolus", "BG"]

df = tester.get_dataframe_from_trace()

print(df)

   time  bolus     BG
0   0.0    0.0   0.00
1   1.0    0.0   3.14
2   2.0    0.0   6.28
3   3.0    0.0   9.42
4   4.0    0.0  12.56
5   5.0    0.0  15.70
6   6.0    0.0  18.84
7   7.0    0.0  21.98
8   8.0    0.0  25.12
9   9.0    0.0  28.26


In [16]:
tester.get_signal("BG")

[np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0),
 np.float64(0.0)]

In [20]:
s = tester.runs[0][1]
print(s)
ss  = tester.get_sample(s)


[1, [3.14], [0], [6.28], 1, False]
